In [2]:
from deepeval.test_case import ConversationalTestCase, LLMTestCase
from deepeval.metrics import ConversationalGEval

from deepeval.test_case import LLMTestCaseParams
import deepeval.models.llms.openai_model as deepeval_models
from deepeval import evaluate
gpt_41_mini = deepeval_models.GPTModel(
    model="gpt-4.1-mini",
    timeout=60,
    #_openai_api_key="sk-BJpqmcWfn3O9bvr2eryf6Uyf1m7VpSQpL7cNxGGlZccRrfA1",
    #base_url="https://open.keyai.shop/v1"
)
json_file = 'conversation_tone.json'


In [ ]:
'''
tc = ConversationalTestCase(
    turns=[
        LLMTestCase(
            input="tui muốn hỏi cách tra cứu hóa đơn đã mua hàng",
            actual_output="Dạ anh có thể thực hiện tra cứu hóa đơn mua hàng tại [đây](https://hddt.fptshop.com.vn/).",
            # Không cần context nếu input đã đủ rõ, nhưng criteria cần xử lý được
            context=["User gender: male"] # Có thể thêm nếu muốn rõ ràng hơn
        ),
        LLMTestCase(
            input="làm sao để tra cứu tình trạng sản phẩm gửi bảo hành tại FPT Shop?",
            actual_output="Dạ anh có thể tra cứu tình trạng máy gửi bảo hành bất cứ lúc nào [Tại đây](https://fptshop.com.vn/kiem-tra-bao-hanh?tab=thong-tin-bao-hanh)..",
            context=["User gender: male"]
        )
    ]
)
'''

In [ ]:
'''
pronoun_consistency_metric = ConversationalGEval(
    name="Vietnamese Pronoun Consistency",
    criteria="""Đánh giá khả năng của chatbot trong việc sử dụng đại từ nhân xưng tiếng Việt một cách chính xác và nhất quán trong suốt cuộc hội thoại. Cụ thể:
    1. Chatbot (assistant) phải LUÔN LUÔN tự xưng là 'em'.
    2. Cách chatbot gọi người dùng (user) phải dựa trên thông tin được cung cấp hoặc cách người dùng tự xưng:
        - Nếu 'User gender' được cung cấp là 'male' hoặc người dùng tự xưng là 'anh', chatbot phải gọi người dùng là 'anh'.
        - Nếu 'User gender' được cung cấp là 'female' hoặc người dùng tự xưng là 'chị', chatbot phải gọi người dùng là 'chị'.
        - Nếu người dùng tự xưng là 'chú', chatbot phải gọi người dùng là 'chú' và tự xưng 'cháu'.
        - Nếu người dùng tự xưng là 'cô', chatbot phải gọi người dùng là 'cô' và tự xưng 'cháu'. (Lưu ý: Cần điều chỉnh nếu model không được fine-tune để xưng 'cháu'. Nếu model luôn xưng 'em', thì tiêu chí này cần phản ánh điều đó, ví dụ: "Nếu người dùng tự xưng là 'chú', chatbot phải gọi người dùng là 'chú' và vẫn tự xưng 'em'").
        - Nếu 'User gender' là 'unknown' hoặc không được cung cấp, và người dùng không tự xưng theo một đại từ cụ thể nào ở trên, chatbot phải gọi người dùng là 'anh/chị'.
    3. Chatbot phải duy trì cách xưng hô đã thiết lập với người dùng một cách nhất quán trong các lượt trả lời tiếp theo, trừ khi có thông tin mới rõ ràng về giới tính hoặc cách xưng hô ưu tiên của người dùng.
    4. Không được sử dụng các cách xưng hô không phù hợp hoặc thiếu tôn trọng.
    """,
    
    evaluation_params=[
        LLMTestCaseParams.INPUT,            # Để phân tích input của user (ví dụ: "anh muốn hỏi...")
        LLMTestCaseParams.ACTUAL_OUTPUT,    # Để phân tích output của model
        # LLMTestCaseParams.CONTEXT,        # Nếu thông tin `User gender` được truyền qua context cho mỗi lượt
    ]
    
)
'''

In [ ]:
import json
from deepeval.test_case import LLMTestCase, ConversationalTestCase
from typing import List
'''load testcase (with actual_output generated or not) from json file'''
def load_testcase_from_json(json_file) -> List[ConversationalTestCase]:
    try:
            #parsed_data = json.loads(generated_json_string)
            # load data from json file
            with open( json_file,"r") as f:
                parsed_data = json.load(f)
            deepeval_test_cases = []
            for conv_data in parsed_data:
                llm_turns = []
                for turn_data in conv_data.get("turns", []):
                    user_input = turn_data.get("user_input")
                    gender_context_val = turn_data.get("user_gender_context")
                    actual_output_placeholder = turn_data.get("bot_actual_output_placeholder", "")  

                    current_context = []
                    if gender_context_val and gender_context_val != "null":  #Kiểm tra null dưới dạng chuỗi
                        current_context.append(f"User gender: {gender_context_val}")
                
                    if user_input:  #Chỉ thêm turn nếu có user_input
                        llm_turns.append(
                            LLMTestCase(
                                input=user_input,
                                actual_output=actual_output_placeholder, # Sẽ được điền sau
                                context=current_context if current_context else None  # DeepEval có thể muốn None nếu context rỗng
                            )
                        )
                if llm_turns:  #Chỉ thêm ConversationalTestCase nếu có turns hợp lệ
                    deepeval_test_cases.append(ConversationalTestCase(turns=llm_turns))  #DeepEval v0.20+ dùng messages
                                                                                        #(hoặc `turns=llm_turns` cho phiên bản cũ hơn)

            print(f"Đã load thành công {len(deepeval_test_cases)} ConversationalTestCase objects.")
            return deepeval_test_cases
    except json.JSONDecodeError as e:
        print(f"Lỗi giải mã JSON: {e}")
    except Exception as e:
        print(f"Lỗi không xác định khi xử lý: {e}")

In [7]:
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import gen_answer
from uuid import uuid4
from models.user import UserRole    
import service.openai as openai_service
from openai.types.chat_model import ChatModel

def get_actual_answer(input: str, gender_context: str,gen_answer_model:ChatModel=None) -> str:
    user = create_user(
        CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
    )
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
    
    openai_service._fine_tuning_model = gen_answer_model # cap nhat tung checkpoint model cho gen_answer
    # Add gender context to the conversation
    history = [
        {"role": "system", "content": "##BASE KNOWLEDGE:\n" + gender_context},
        {"role": "user", "content": str(input)}
    ]

    return gen_answer(
        thread_id=thread.id,
        history=history,
        user_id=user.id,
    )

def lst_context_to_str(lst: list) -> str:
    turn_str = ""
    for t in lst:
        turn_str += t
    return turn_str

# ko luu conversation vao history
def get_actual_answer_for_all_testcase_without_conversation_history(json_file:str,gen_answer_model:ChatModel=None,iter_checkpoint:int=0):
    #deepeval_test_cases = load_testcase_from_json(json_file) # load test case with actual_output werent generated
    '''
    get_actual_answer for all testcase
    iter_checkpoint is the index of the checkpoint fine-tuned model
    gen_answer_model is the fine-tuned model to use for generating responses
    '''
    conversations = load_testcase_from_json(json_file)
    process_conversations = []
    id = 0
    for conversational_test_case in conversations:
        processed_turns = []
        for turn in conversational_test_case.turns:
            turn.actual_output = get_actual_answer(turn.input, lst_context_to_str(turn.context),gen_answer_model)
            print(turn.input,turn.actual_output)
            processed_turn = {
                "user_input": turn.input,
                "user_gender_context": turn.context,
                "bot_actual_output_placeholder": turn.actual_output
            }
            processed_turns.append(processed_turn)

        process_conversations.append({
            "conversation_id": conversational_test_case.id,
            "turns": processed_turns
        })
        id += 1
        '''write to json file'''
        print(f"Processed {id} conversations with actual answers.")
    with open(f"conversation_tone_{iter_checkpoint}.json", "w") as f:
        json.dump(process_conversations, f)

#luu conversation vao history
def get_actual_answer_with_conversation_history(json_file: str, gen_answer_model:ChatModel=None,iter_checkpoint:int=0):
    
    conversations = load_testcase_from_json(json_file)
    process_conversations = []
    id = 0
    openai_service._fine_tuning_model = gen_answer_model # cap nhat tung checkpoint model cho gen_answer

    for conv in conversations:
        processed_turns = []
        turns = conv.turns
        user = create_user(
            CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
        )
        thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
    
        # Initialize conversation history with system message
        history = [
            {"role": "system", "content": "##BASE KNOWLEDGE:\n" + turns[0]["user_gender_context"]}
        ]
        
        for turn in turns:
            # Add user input to history
            history.append({"role": "user", "content": turn["user_input"]})
            
            # Set the model for this conversation
            #openai_service._fine_tuning_model = gen_answer_model
            
            # Generate bot response using accumulated history
            bot_response = gen_answer(
                #thread_id=uuid4(),  # Generate new thread ID for each turn
                thread_id=thread.id,
                history=history,
                #user_id=uuid4(),  # Generate new user ID for each turn
                user_id=user.id,
            )
            turn.actual_output = bot_response
            # Add bot response to history for next turn
            history.append({"role": "assistant", "content": bot_response})
            
            # Store processed turn
            
            processed_turn = {
                "user_input": turn.input,
                "user_gender_context": turn.context,
                "bot_actual_output_placeholder": turn.actual_output
            }
            processed_turns.append(processed_turn)
        
        process_conversations.append({
            "conversation_id": conv.id,
            "turns": processed_turns
        })
        id += 1
        print(f"Processed {id} conversations with actual answers.")
    with open(f"{json_file}_{iter_checkpoint}.json", "w") as f:
        json.dump(process_conversations, f)


In [ ]:
import json


In [ ]:
''' function to get actual_output for all testcase and write to json file'''
'''
def get_actual_output_for_all_testcase(deepeval_test_cases,gen_answer_model,iter_checkpoint):
    for conversational_test_case in deepeval_test_cases:
        for turn in conversational_test_case.turns:
            turn.actual_output = get_actual_answer(turn.input, lst_context_to_str(turn.context),gen_answer_model)
            print(turn.input,turn.actual_output)
        with open(f"conversation_tone_{iter_checkpoint}.json", "w") as f:
            json.dump(conversational_test_case, f)
'''            

In [4]:
pronoun_consistency_metric = ConversationalGEval(
    name="Vietnamese Pronoun Consistency",
    criteria="""Đánh giá khả năng của chatbot trong việc sử dụng đại từ nhân xưng tiếng Việt một cách chính xác và nhất quán trong suốt cuộc hội thoại. Cụ thể:
    1. Chatbot (assistant) trong 'actual_output' **nên** tự xưng là 'em'. Việc này được ưu tiên, đặc biệt ở các lượt nói đầu hoặc khi cần làm rõ hành động của chatbot (ví dụ: 'Dạ em chào anh', 'Em có thể giúp gì ạ'). Việc không xưng 'em' trong một số câu trả lời ngắn và trực tiếp ở các lượt sau có thể được chấp nhận nếu văn phong tự nhiên và vai trò của chatbot đã rõ ràng từ trước. Chatbot **TUYỆT ĐỐI KHÔNG** được dùng 'tôi', 'mình' để tự xưng.
    2. Cách chatbot gọi người dùng (user) phải dựa trên thông tin được cung cấp hoặc cách người dùng tự xưng:
        - Nếu 'User gender' được cung cấp là 'male' hoặc người dùng tự xưng là 'anh' như 'anh muốn hỏi...', chatbot phải gọi người dùng là 'anh'.
        - Nếu 'User gender' được cung cấp là 'female' hoặc người dùng tự xưng là 'chị' như 'chị muốn hỏi...', chatbot phải gọi người dùng là 'chị'.
        - Nếu người dùng tự xưng là 'chú', chatbot phải gọi người dùng là 'chú' và tự xưng 'cháu'.
        - Nếu người dùng tự xưng là 'bác', chatbot phải gọi người dùng là 'bác' và tự xưng 'cháu'.
        - Nếu người dùng tự xưng là 'cô', chatbot phải gọi người dùng là 'cô' và tự xưng 'cháu'.
        - Nếu 'User gender' được cung cấp là 'unknown' hoặc không được cung cấp, và người dùng không tự xưng theo một đại từ cụ thể nào ở trên, chatbot phải gọi người dùng là 'anh/chị'.

    3. Tính nhất quán: Chatbot phải duy trì cách xưng hô đã được thiết lập với người dùng một cách nhất quán trong các lượt trả lời tiếp theo trong cùng một cuộc hội thoại, trừ khi có thông tin mới rõ ràng thay đổi cách xưng hô.
    4. Không được sử dụng các cách xưng hô không phù hợp hoặc thiếu tôn trọng.

    HƯỚNG DẪN CHẤM ĐIỂM:
    - Điểm 1.0: Tuân thủ hoàn hảo tất cả các quy tắc trên trong mọi lượt của hội thoại.
    - **Phạt nặng (điểm gần 0): Nếu chatbot tự xưng sai hoàn toàn (ví dụ: dùng 'tôi', 'mình' thay vì 'em'; hoặc dùng sai giữa 'em' và 'cháu' khi ngữ cảnh yêu cầu rõ ràng).**
    - Phạt nặng (điểm gần 0): Nếu chatbot gọi sai người dùng một cách rõ ràng (ví dụ: context là 'User gender: male' nhưng chatbot gọi là 'chị').
    - **Phạt nhẹ hoặc không phạt: Đối với trường hợp chatbot không tự xưng 'em' trong một số ít câu trả lời ngắn, trực tiếp ở các lượt sau, khi việc này không ảnh hưởng đáng kể đến sự rõ ràng, tính lịch sự của cuộc hội thoại và vai trò chatbot đã được thiết lập.**
    - Xem xét toàn bộ cuộc hội thoại để đánh giá tính nhất quán.
    """,
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.CONTEXT,
    ],
    model=gpt_41_mini,
)

In [6]:
pronoun_consistency_metric = ConversationalGEval(
    name="Vietnamese Pronoun Consistency",
    criteria="""Đánh giá khả năng của chatbot trong việc sử dụng đại từ nhân xưng tiếng Việt một cách chính xác và nhất quán trong suốt cuộc hội thoại. 

    THÔNG TIN CẦN PHÂN TÍCH:
    - 'input': Tin nhắn của người dùng - cần phân tích cách người dùng tự xưng và ngữ cảnh
    - 'actual_output': Phản hồi của chatbot - cần đánh giá cách chatbot tự xưng và gọi người dùng
    - 'context': Thông tin bổ sung về người dùng (ví dụ: User gender) và ngữ cảnh cuộc hội thoại

    QUY TẮC ĐÁNH GIÁ:

    1. **Cách chatbot tự xưng trong 'actual_output':**
       - Chatbot **nên** tự xưng là 'em' (được ưu tiên, đặc biệt ở các lượt nói đầu hoặc khi cần làm rõ hành động)
       - Chatbot **TUYỆT ĐỐI KHÔNG** được dùng 'tôi', 'mình' để tự xưng
       - Ngoại lệ: Khi người dùng trong 'input' tự xưng là 'chú', 'bác', 'cô' thì chatbot phải tự xưng 'cháu'

    2. **Cách chatbot gọi người dùng (dựa trên 'input' và 'context'):**
       - Nếu trong 'context' có 'User gender: male' HOẶC người dùng trong 'input' tự xưng 'anh' → chatbot gọi 'anh'
       - Nếu trong 'context' có 'User gender: female' HOẶC người dùng trong 'input' tự xưng 'chị' → chatbot gọi 'chị'
       - Nếu người dùng trong 'input' tự xưng 'chú' → chatbot gọi 'chú' và tự xưng 'cháu'
       - Nếu người dùng trong 'input' tự xưng 'bác' → chatbot gọi 'bác' và tự xưng 'cháu'
       - Nếu người dùng trong 'input' tự xưng 'cô' → chatbot gọi 'cô' và tự xưng 'cháu'
       - Nếu trong 'context' có 'User gender: unknown' hoặc không có thông tin gender, và người dùng trong 'input' không tự xưng rõ ràng → chatbot gọi 'anh/chị'

    3. **Tính nhất quán:** 
       - Phân tích toàn bộ lịch sử hội thoại trong 'context' để đảm bảo chatbot duy trì cách xưng hô đã thiết lập
       - Chỉ chấp nhận thay đổi cách xưng hô khi có thông tin mới rõ ràng trong 'input'

    4. **Tính lịch sự và phù hợp:**
       - Không sử dụng cách xưng hô thiếu tôn trọng hoặc không phù hợp với văn hóa Việt Nam

    HƯỚNG DẪN CHẤM ĐIỂM:
    - **Điểm 1.0:** Tuân thủ hoàn hảo tất cả các quy tắc trên, phù hợp với thông tin trong 'input', 'context' và thể hiện tính nhất quán trong 'actual_output'
    
    - **Phạt nặng (điểm 0.0-0.3):**
      • Chatbot tự xưng sai hoàn toàn ('tôi', 'mình' thay vì 'em')
      • Sử dụng sai giữa 'em' và 'cháu' khi 'input' hoặc 'context' yêu cầu rõ ràng
      • Gọi sai người dùng rõ ràng (ví dụ: 'context' là 'User gender: male' nhưng gọi 'chị')
      • Không nhất quán trong cùng cuộc hội thoại mà không có lý do chính đáng từ 'input' mới
    
    - **Phạt vừa (điểm 0.4-0.7):**
      • Thiếu một số cách xưng hô cần thiết ở những vị trí quan trọng
      • Không tận dụng đầy đủ thông tin từ 'context' để xác định cách xưng hô phù hợp
    
    - **Phạt nhẹ hoặc không phạt (điểm 0.8-1.0):**
      • Chatbot không tự xưng 'em' trong một số ít câu trả lời ngắn, trực tiếp ở các lượt sau, khi vai trò đã được thiết lập và không ảnh hưởng đến tính lịch sự

    CÁCH PHÂN TÍCH:
    1. Đọc kỹ 'context' để hiểu thông tin về người dùng và lịch sử hội thoại
    2. Phân tích 'input' để xác định cách người dùng tự xưng và mong đợi được gọi
    3. Đánh giá 'actual_output' dựa trên các quy tắc trên
    4. Xem xét tính nhất quán với các lượt hội thoại trước đó (nếu có trong context)
    """,
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.CONTEXT,
    ],
    model=gpt_41_mini,
)

In [7]:
test_case = ConversationalTestCase(
    turns=[
        LLMTestCase(
            input="Chị muốn hỏi cách tra cứu điểm mua hàng đã tích được tại FPT Shop .",
            actual_output= " Chị có thể thực hiện tra cứu điểm tích [tại đây](https://fptshop.com.vn/tai-khoan/lich-su-tich-diem) bằng cách đăng nhập số điện thoại mua hàng của chị.",
            # Không cần context nếu input đã đủ rõ, nhưng criteria cần xử lý được
            # context=["User gender: female"] # Có thể thêm nếu muốn rõ ràng hơn
            context=["User gender: unknown"]
        ),
        LLMTestCase(
            input="Vậy còn tra cứu về thông tin trúng thưởng của FPT Shop khi tham gia các chương trình mini game?",
            actual_output="Chị có thể thực hiện tra cứu [tại đây](https://fptshop.com.vn/khuyen-mai/thong-tin-trao-thuong)",
            context=["User gender: unknown"]
        )
    ]
)
pronoun_consistency_metric.measure(test_case)
print(pronoun_consistency_metric.score, pronoun_consistency_metric.reason)


Output()

0.8029362452578461 The chatbot consistently uses 'Chị' to address the user, matching the user's self-reference in the input. Although the user's gender is unknown in context, the chatbot maintains politeness and cultural appropriateness without inconsistent shifts in pronouns.


In [6]:

'''tc = deepeval_test_cases[0]
for turn in tc.turns:
    print(turn.input)
    # turn list to str
    turn_str = ""
    for t in turn.context:
        turn_str += t
    print(turn_str)
'''

Anh muốn hỏi cách tra cứu về hóa đơn đã mua hàng tại FPT Shop?
User gender: male
Nếu hóa đơn đó mua hơn 1 năm rồi thì có tìm lại được không shop?
User gender: male


In [9]:

'''
for turn in tc.turns:
    turn.actual_output = get_actual_answer(turn.input, lst_context_to_str(turn.context))
    print(turn.actual_output)
'''

🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01972b64-165e-7c72-bfd0-d513b251ee06
2025-06-01 12:07:50 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
User request: {'user_demand': <ProductType.UNDETERMINED: 'undetermined'>, 'user_info': {'phone_number': None, 'email': None}}
Detect demand response: type='finished' content='The user request has been successfully processed.' instructions=[] UserIntent(is_user_needs_other_suggestions=False, product_type=None)
2025-06-01 12:07:50 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
faq id : 12
faq id : 16
faq id : 62
faq id : 18
2025-06-01 12:07:59 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Updating user memory with id: f6eb87ad-616a-4b58-a7e3-126f60c8f886 and data: {'user_demand': None, 'product_name': None, 'brand_code': None, 'brand_name': None, 'min_price': None, 'max_price': None, 'phone_number': None, 'email': None, 'intent': {'is_user_need

In [10]:
# danh gia rieng le 1 testcase
pronoun_consistency_metric.measure(tc)
print(pronoun_consistency_metric.score, pronoun_consistency_metric.reason)

Output()

🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01972b64-f9fa-7eb3-9844-365579882529

2025-06-01 12:08:55 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01972b65-1a1a-7400-b29b-61768612942a

2025-06-01 12:08:57 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


1.0 The chatbot consistently uses 'em' for self-reference and addresses the male user as 'anh' in both turns, matching the user gender in context. The form of address remains consistent and respectful throughout the conversation.


In [ ]:

        

'''save the output of get_actual_answer for all testcase to json file for each model'''


def eval_tc(_testcases,_metrics,iter_checkpoint):
    evaluation_results = evaluate(
        test_cases=_testcases,
        metrics=_metrics,
    )
    # điểm số của metric được lưu trong mỗi đối tượng test case sau khi evaluate
    score = 0
    for result in evaluation_results: # evaluate() trả về list các test cases đã được cập nhật
        print(f"Test Case ID: {result.id if result.id else 'N/A'}") # ConversationalTestCase có thể không có id trừ khi bạn set
        for metric_result in result.metrics: # Mỗi test case có thể có nhiều metric
            if metric_result.name == pronoun_consistency_metric.name:
                print(f"  Metric: {metric_result.name}")
                print(f"  Score: {metric_result.score}")
                if metric_result.reason: 
                    print(f"  Reason: {metric_result.reason[:300]}...") # In một phần lý do
                print("-" * 20)
                score += metric_result.score
    mean_score = score/len(evaluation_results)
    print(f"Mean score: {mean_score}")
            
    
    # get the mean score of all testcase
    
    #mean_score = evaluation_results.score.mean()
    # save result to file
    with open(f"evaluation_results_{iter_checkpoint}.json", "w") as f:
        json.dump(evaluation_results, f)
        json.dump(mean_score, f)
    
    




In [ ]:
checkpoint_lst = ["ft:gpt-4o-mini-2024-07-18:personal::Bdao6dn4:ckpt-step-50","ft:gpt-4o-mini-2024-07-18:personal::Bdao6ShS:ckpt-step-100","ft:gpt-4o-mini-2024-07-18:personal::Bdao7Wp2"]

def get_actual_answer_for_all_testcase_for_all_checkpoint(checkpoint_lst):
    for iter_checkpoint,checkpoint_model in enumerate(checkpoint_lst):
        print(checkpoint_model)
        # load checkpoint
        model = checkpoint_model

        # 1 trong 2 cach de luu actual_output vao json file moi
        #get_actual_answer_for_all_testcase_without_conversation_history(json_file,model,iter_checkpoint)
        get_actual_answer_with_conversation_history(json_file,model,iter_checkpoint)





In [ ]:
get_actual_answer_for_all_testcase_for_all_checkpoint(checkpoint_lst)

In [ ]:
"""Thay vi chi dùng checkpoint cuoi của fine-tuned model , dùng toàn bộ checkpoint để đánh giá"""
def eval_all_checkpoint(deepeval_test_cases,checkpoint_lst):
    for iter_checkpoint,checkpoint_model in enumerate(checkpoint_lst):
        load_testcase_from_json(f'{json_file}_{iter_checkpoint}.json')
        print(checkpoint_model)
        # load checkpoint
        # evaluate
        eval_tc(deepeval_test_cases,[pronoun_consistency_metric],iter_checkpoint)

In [ ]:
# danh gia toan bo testcase
#eval_all_checkpoint(deepeval_test_cases,checkpoint_lst)

In [ ]:

history = [
    {"role": "system", "content": "##BASE KNOWLEDGE:\n" +"User gender: unknown"}
]
turns = [
    {"user_input": "Chú muốn hỏi Shop có hỗ trợ trả góp qua Home PayLater không?"},
    {"user_input": "Vậy cửa hàng có hỗ trợ trả góp qua Kredivo không?"}
]
user = create_user(
            CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user)
        )
thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
for turn in turns:
    # Add user input to history
    history.append({"role": "user", "content": turn["user_input"]})
    
    # Set the model for this conversation
    #openai_service._fine_tuning_model = gen_answer_model
    
    # Generate bot response using accumulated history
    bot_response = gen_answer(
        #thread_id=uuid4(),  # Generate new thread ID for each turn
        thread_id=thread.id,
        history=history,
        #user_id=uuid4(),  # Generate new user ID for each turn
        user_id=user.id,
    )
    turn.actual_output = bot_response
    # Add bot response to history for next turn
    history.append({"role": "assistant", "content": bot_response})